In [1]:
import pandas as pd

In [47]:
df = pd.read_csv("data/ma-airports.csv")
df.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated'],
      dtype='object')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45 non-null     int64  
 1   ident              45 non-null     object 
 2   type               45 non-null     object 
 3   name               45 non-null     object 
 4   latitude_deg       45 non-null     float64
 5   longitude_deg      45 non-null     float64
 6   elevation_ft       39 non-null     float64
 7   continent          45 non-null     object 
 8   country_name       45 non-null     object 
 9   iso_country        45 non-null     object 
 10  region_name        45 non-null     object 
 11  iso_region         45 non-null     object 
 12  local_region       45 non-null     object 
 13  municipality       44 non-null     object 
 14  scheduled_service  45 non-null     int64  
 15  gps_code           29 non-null     object 
 16  iata_code          21 non-nu

In [49]:
df['last_updated'] = pd.to_datetime(df['last_updated'])

In [52]:
datetime_dim = df[['last_updated']].drop_duplicates().reset_index(drop=True)
datetime_dim['update_hour'] = datetime_dim['last_updated'].dt.hour
datetime_dim['update_day'] = datetime_dim['last_updated'].dt.day
datetime_dim['update_month'] = datetime_dim['last_updated'].dt.month
datetime_dim['update_year'] = datetime_dim['last_updated'].dt.year
datetime_dim['update_weekday'] = datetime_dim['last_updated'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index+1

In [54]:
airport_coordinates_dim = df[['latitude_deg','longitude_deg','elevation_ft']].drop_duplicates().reset_index(drop=True)
airport_coordinates_dim['coordinates_id'] = airport_coordinates_dim.index+1

In [57]:
airport_coordinates_dim['elevation_ft'] = airport_coordinates_dim['elevation_ft'].fillna(0)


In [59]:
regions_dim = df[['region_name','local_region']].drop_duplicates().reset_index(drop=True)
regions_dim['region_id'] = regions_dim.index+1


cities_dim = df.merge(regions_dim,on=['region_name','local_region'])[['municipality','region_id']].drop_duplicates().reset_index(drop=True)
cities_dim['city_id']  =cities_dim.index+1

In [63]:
airport_type_dim = df[['type']].drop_duplicates().reset_index(drop=True)
airport_type_dim['type_id'] = airport_type_dim.index+1


In [65]:
fact_table = df.merge(datetime_dim, on='last_updated').merge(airport_coordinates_dim,on=['latitude_deg','longitude_deg']).merge(cities_dim,on='municipality').merge(airport_type_dim,on='type')[['name','ident','scheduled_service','score','type_id','city_id','coordinates_id','datetime_id']]

In [66]:
fact_table

,name,ident,scheduled_service,score,type_id,city_id,coordinates_id,datetime_id
0,Menara Airport,GMMX,1,750,1,1,1,1
1,Al Massira Airport,GMAD,1,750,1,6,3,3
2,Tangier Ibn Battuta Airport,GMTT,1,500,1,9,4,4
3,Rabat-Salé Airport,GMME,1,750,1,10,5,5
4,Fes Saïss International Airport,GMFF,1,750,1,11,6,6
5,Oujda Angads Airport,GMFO,1,750,1,13,7,7
6,Ouarzazate Airport,GMMZ,1,750,1,14,8,8
7,Essaouira-Mogador Airport,GMMI,1,500,1,15,9,9
8,Nador Al Aaroui International Airport,GMMW,1,750,1,16,10,10
9,Cherif Al Idrissi Airport,GMTA,1,750,1,18,11,11
